In [1]:
import json
import torch
import numpy as np
from sklearn.preprocessing import OneHotEncoder

import sys
import warnings
warnings.filterwarnings('ignore')

# Path to local library
sys.path.append('./models')

# Local library
from util import load_data, normalise, train_validate, test

# Random seed
params = json.load(open("hyperparams.json", "r"))
NJOBS = params['njobs']
RANDOM_STATE = params['random_state']
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed(RANDOM_STATE) 
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cuda:0 device


In [2]:
rssi_train, accl_train, location_train = load_data(params['data_path'], control=True)
rssi_test, accl_test, location_test = load_data(params['data_path'], control=False)

train_idx, validation_idx = np.array([]).astype(np.int64), np.array([]).astype(np.int64)
for room in ['dining_room', 'living_room', 'kitchen', 'hall', 'stairs', 'porch/outside_front_door']:
    temp = np.unique(np.argwhere(location_train == room)[:, 0])
    if len(temp):
        train_idx = np.concatenate([train_idx, temp[:int(len(temp)/2)]])
        validation_idx = np.concatenate([validation_idx, temp[int(len(temp)/2):]])
    else:
        print(f'Data for ({room}) is not available!')

rssi_scaler, rssi_train, rssi_test, rssi_val = normalise(rssi_train[train_idx], rssi_test, rssi_train[validation_idx])
accl_scaler, accl_train, accl_test, accl_val = normalise(accl_train[train_idx], accl_test, accl_train[validation_idx])

encoder = OneHotEncoder(sparse=False)
encoder.fit(np.unique(np.concatenate(
    [location_train[train_idx], location_train[validation_idx], location_test]
)).reshape(-1, 1))
location_val = encoder.transform(location_train[validation_idx].reshape(-1, 1)).reshape(len(validation_idx), location_train.shape[1], -1)
location_train = encoder.transform(location_train[train_idx].reshape(-1, 1)).reshape(len(train_idx), location_train.shape[1], -1)
location_test = encoder.transform(location_test.reshape(-1, 1)).reshape(rssi_test.shape[0], rssi_test.shape[1], -1)

print('Train data: {}, Validation data: {}, Test data: {}'.format(
    rssi_train.shape, rssi_val.shape, rssi_test.shape
))

Data for (stairs) is not available!
Data for (porch/outside_front_door) is not available!
Train data: (19722, 25, 20), Validation data: (19725, 25, 20), Test data: (39566, 25, 20)


In [3]:
model = train_validate(
    torch.from_numpy(rssi_train).type(torch.float).to(device), 
    torch.from_numpy(accl_train).type(torch.float).to(device), 
    torch.from_numpy(location_train).type(torch.float).to(device),
    torch.from_numpy(rssi_val).type(torch.float).to(device),
    torch.from_numpy(accl_val).type(torch.float).to(device),
    torch.from_numpy(location_val).type(torch.float).to(device), 
    np.argmax(encoder.transform([['hall']])), params
)

Batch: 1, Train Loss: 29.99791, Acc: 2.43383
Batch: 6, Train Loss: 6.99540, Acc: 28.39469
Batch: 11, Train Loss: 6.76656, Acc: 53.22483
Batch: 16, Train Loss: 25.71478, Acc: 53.22990
Val Acc: 55.88340
Batch: 1, Train Loss: 5.79832, Acc: 5.12118
Batch: 6, Train Loss: 4.17212, Acc: 31.08204
Batch: 11, Train Loss: 5.58911, Acc: 55.91218
Batch: 16, Train Loss: 13.41573, Acc: 66.76301
Val Acc: 93.15082
Batch: 1, Train Loss: 8.09753, Acc: 5.12118
Batch: 6, Train Loss: 6.36520, Acc: 31.06683
Batch: 11, Train Loss: 7.25826, Acc: 55.84119
Batch: 16, Train Loss: 9.72239, Acc: 77.89271
Val Acc: 93.54119
Batch: 1, Train Loss: 7.75442, Acc: 5.11611
Batch: 6, Train Loss: 5.24039, Acc: 31.06176
Batch: 11, Train Loss: 5.92957, Acc: 55.88683
Batch: 16, Train Loss: 7.86231, Acc: 77.98905
Val Acc: 93.57161
Batch: 1, Train Loss: 5.94779, Acc: 5.12118
Batch: 6, Train Loss: 3.85104, Acc: 31.08204
Batch: 11, Train Loss: 4.63049, Acc: 55.90711
Batch: 16, Train Loss: 6.33558, Acc: 78.00933
Val Acc: 93.59189
Ba

In [4]:
test(
    model, 
    torch.from_numpy(rssi_test).type(torch.float).to(device), 
    torch.from_numpy(accl_test).type(torch.float).to(device), 
    torch.from_numpy(location_test).type(torch.float).to(device), 
    params
)

Test error: 
 Accuracy: 94.7%
